<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [15]:
-- What is the oldest business on each continent?
SELECT DISTINCT co.continent, Min(co.country) as country, Min(ca.category) as business, 
		MIN(b.year_founded) as year_founded
FROM countries as co
LEFT JOIN businesses as b
ON co.country_code = b.country_code
LEFT JOIN categories as ca
ON b.category_code = ca.category_code
WHERE b.year_founded IS NOT NULL
GROUP BY co.continent
ORDER BY year_founded ASC;

,continent,country,business,year_founded
0,Asia,Afghanistan,Agriculture,578
1,Europe,Albania,Agriculture,803
2,North America,Barbados,Agriculture,1534
3,South America,Argentina,Banking & Finance,1565
4,Africa,Algeria,Agriculture,1772
5,Oceania,Australia,Banking & Finance,1809


![Task 1_Which Continents Host the Worlds Oldest Businesses](Task%201_Which%20Continents%20Host%20the%20Worlds%20Oldest%20Businesses.png)


In [17]:
-- How many countries per continent lack data on the oldest businesses
-- Does including the `new_businesses` data change this? 

WITH all_businesses AS (
  SELECT country_code
  FROM businesses
  UNION
  SELECT country_code
  FROM new_businesses
)

SELECT 
  c.continent,
  COUNT(DISTINCT c.country_code) AS countries_without_businesses
FROM countries c
LEFT JOIN all_businesses ab ON c.country_code = ab.country_code
WHERE ab.country_code IS NULL
GROUP BY c.continent;

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


![Task 2_Where Are the Gaps - Missing Historical Business Data Worldwide](Task%202_Where%20Are%20the%20Gaps%20-%20Missing%20Historical%20Business%20Data%20Worldwide.png)


In [42]:
WITH ages AS (
  SELECT
    ca.category,
    (EXTRACT(YEAR FROM CURRENT_DATE)::int - b.year_founded) AS age
  FROM businesses b
  JOIN categories ca ON ca.category_code = b.category_code
  WHERE b.year_founded IS NOT NULL
),
per_category AS (
  SELECT
    category,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY age)       AS p90_age,
    AVG( (age >= 200)::int )                               AS share_200y,
    MAX(age)                                               AS max_age,
    COUNT(*)                                               AS n_businesses
  FROM ages
  GROUP BY category
),
ranked AS (
  SELECT
    *,
    RANK() OVER (ORDER BY p90_age DESC)     AS r_p90,
    RANK() OVER (ORDER BY share_200y DESC)  AS r_share200,
    RANK() OVER (ORDER BY max_age DESC)     AS r_max
  FROM per_category

),
scored AS (
  SELECT
    category,
    p90_age, share_200y, max_age,
    (r_p90 + r_share200 + r_max) / 3.0 AS avg_rank
  FROM ranked
)
SELECT
  category,
  (EXTRACT(YEAR FROM CURRENT_DATE)::int - max_age) AS oldest_year
FROM scored
ORDER BY oldest_year ASC
LIMIT 15;

,category,oldest_year
0,Construction,578
1,"Cafés, Restaurants & Bars",803
2,"Distillers, Vintners, & Breweries",862
3,Manufacturing & Production,864
4,Agriculture,1218
5,Tourism & Hotels,1230
6,Mining,1248
7,Medical,1422
8,Postal Service,1520
9,Banking & Finance,1565


![Task 3_Pioneers of Industry - The Oldest Businesses Around the World.png](Task%203_Pioneers%20of%20Industry%20-%20The%20Oldest%20Businesses%20Around%20the%20World.png.png)
